<a href="https://colab.research.google.com/github/deegee45/Quant_Fin/blob/main/Asset_Pricing_Models(Crude).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Asset Pricing Models

In [1]:
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
from scipy import stats


Assumptions of Asset Pricing Models:
1. Linearity
2. Perfect Information -
3. Efficient Market - Any information is reflected immediately in the security
   price




Generalized  assset pricing model:-
Regression of Expected Return on Factor that impacts the expected return

E[r] = a + bF + c

where a is the intercept term - representing the expected return when factor F = 0

Factor F - F represents the factor that causes the E[r] to change
c - error

### Capital Asset Pricing Model - (CAPM)


CAPM works on the premise that the return on any asset is based on its relationship with the market (Index).

However in theory, the market portfolio of CAPM implies every  asset in the world.

CAPM ignores every other thing - firm values, fundamentals, etc.

This is ultimately because CAPM focusses on diversification

If the market is perfect, a==e==0

E[r] - rf = b(E[rm] - rf)

where E[rm] - rf implies excess market returns

In [2]:
def calc_beta(ticker):

  """
  Calculate the beta of the given ticekr using an OLS regression.
  Benchmark index - NIFTY
  """

  # get the data from yfinance
  df1 = yf.download("^NSEI",period="1y")
  df2 = yf.download(ticker,period="1y")

  # calculate daily returns
  df1['NSEr'] = df1["Adj Close"].pct_change(1)
  df2['IFBr'] = df2["Adj Close"].pct_change(1)

  # combined returns dataframe
  df=pd.concat([df1['NSEr'],df2['IFBr']],axis=1)

  # drop first missing row
  data = df.dropna(axis=0)

  # regression ols
  slope, intercept, r_value, p_value, std_err = stats.linregress(data.NSEr,data.IFBr)

  return slope



In [3]:
def expected_return(ticker, annualised=True, annualise_method='sophisticated'):
    """
    Returns the expected return of a security given price data.
    """

    # Calculate returns of prices
    data= yf.download(ticker,period='1y')
    returns = data['Adj Close'].pct_change(1)

    # Calculate the expected return using the mean method
    expected_return_daily = returns.mean()

    if annualised:
        if annualise_method == 'sophisticated':
            expected_return_annual = ((1 + expected_return_daily) ** 250) - 1
        elif annualise_method == 'crude':
            # Crude method
            expected_return_annual = expected_return_daily * 250

        return expected_return_annual

    else:
        return expected_return_daily

In [4]:
beta = calc_beta("IDFCFIRSTB.NS")
# Risk free rate is assumed to be the 10y bondd yield in India
rf = 0.077071
rm = expected_return("^NSEI")

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


CAPM

E<sub>r</sub> = r<sub>f</sub>+ beta*(E<sub>m</sub> - r<sub>f</sub>)

In [5]:
er = rf + beta*(rm-rf)

In [6]:
# expected returns to be earned
print(rf*100)

7.7071000000000005


### Five factor Fama-French model

The Fama-French is an arbitrage based asse pricing model. The model implies that the expected return is dependent on multiple factors. The five factor fama french model lists the following:

1. Market Risk (RMRF)
2. Size (SMB)
3. Profitability (RMW)
4. Investment (CMA)
5. Value(HML)


Ri = αi + βi(RMRF) + si(SMB) + hi(HML) + ri(RMW) + ci(CMA) + εi

Data Source = [Click Here](https://faculty.iima.ac.in/~iffm/Indian-Fama-French-Momentum/)

In [7]:
factors_data = pd.read_csv('/content/2023-03_FourFactors_and_Market_Returns_Monthly_SurvivorshipBiasAdjusted.csv')

In [19]:
factors_data.tail()

,Date,SMB,HML,WML,MF,RF
349,2022-11,-0.466224,5.788724,-1.513231,2.301874,0.515031
350,2022-12,1.678694,3.165031,-0.085832,-3.980242,0.511031
351,2023-01,0.377754,3.649467,-3.265305,-4.549322,0.544402
352,2023-02,-2.319557,0.038092,-1.773989,-5.461938,0.497016
353,2023-03,-2.514654,1.172053,1.264843,0.230031,0.565059


In [9]:
updated_factors_data=factors_data[231:]

In [10]:
ticker_data = yf.download("TANLA.NS",start='2013-01-01',end='2023-03-31',interval='1mo')

[*********************100%***********************]  1 of 1 completed


In [11]:
ticker_data

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2013-01-01,5.800000,6.550000,5.050000,5.300000,5.035854,2807987
2013-02-01,5.500000,5.800000,3.950000,4.250000,4.038184,1507844
2013-03-01,4.350000,4.900000,3.000000,3.150000,2.993007,1382342
2013-04-01,3.000000,5.800000,3.000000,4.150000,3.943168,2028495
2013-05-01,4.150000,4.350000,3.450000,3.550000,3.373071,765015
...,...,...,...,...,...,...
2022-11-01,748.000000,849.000000,720.000000,791.650024,791.650024,10393348
2022-12-01,797.000000,818.450012,651.099976,714.450012,714.450012,5086159
2023-01-01,716.700012,754.950012,625.000000,646.799988,646.799988,5240781


In [12]:
ticker_data['monthly_returns'] = (ticker_data.Close)/(ticker_data.Open) -1

In [13]:
ticker_data.head()

,Open,High,Low,Close,Adj Close,Volume,monthly_returns
Date,,,,,,,
2013-01-01,5.80,6.55,5.05,5.30,5.035854,2807987,-0.086207
2013-02-01,5.50,5.80,3.95,4.25,4.038184,1507844,-0.227273
2013-03-01,4.35,4.90,3.00,3.15,2.993007,1382342,-0.275862
2013-04-01,3.00,5.80,3.00,4.15,3.943168,2028495,0.383333
2013-05-01,4.15,4.35,3.45,3.55,3.373071,765015,-0.144578


In [14]:
excess_returns=np.array(ticker_data['monthly_returns']) - np.array(updated_factors_data['RF'])

In [16]:
ticker_data['excess_monthy_returns'] = excess_returns

In [20]:
max(ticker_data['excess_monthy_returns'])

0.6381693348628787

In [27]:
import statsmodels.api as sm
y=ticker_data['excess_monthy_returns']

# Independent variables: Factor returns
X = updated_factors_data[['SMB', 'HML', 'WML', 'MF']]

# Add a constant column for the intercept term
X = sm.add_constant(X)

# drop index
y = y.reset_index(drop=True)
X = X.reset_index(drop=True)

# Perform the multiple linear regression
model = sm.OLS(y, X)
results = model.fit()

In [29]:
print(results.summary())


                              OLS Regression Results                             
Dep. Variable:     excess_monthy_returns   R-squared:                       0.188
Model:                               OLS   Adj. R-squared:                  0.160
Method:                    Least Squares   F-statistic:                     6.808
Date:                   Fri, 14 Jul 2023   Prob (F-statistic):           5.76e-05
Time:                           16:53:02   Log-Likelihood:                -3.4677
No. Observations:                    123   AIC:                             16.94
Df Residuals:                        118   BIC:                             31.00
Df Model:                              4                                         
Covariance Type:               nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.4722 